# Preparing training data for the BERT model

In [1]:
import pandas as pd
import sqlite3
import os
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

Defining user names just as in the ClassApp

In [3]:
user_list = ['GV', 'NA', 'RC', 'WZ', 'EM', 'FR', 'HC', 'JF', 'MG']
db_names = ['data_' + user + '.db' for user in user_list]
db_names

['data_GV.db',
 'data_NA.db',
 'data_RC.db',
 'data_WZ.db',
 'data_EM.db',
 'data_FR.db',
 'data_HC.db',
 'data_JF.db',
 'data_MG.db']

Creating paths to their .db files in the classapp_output folder

In [4]:
db_paths = [os.path.join('classapp_output', db_name) for db_name in db_names]
db_paths

['classapp_output\\data_GV.db',
 'classapp_output\\data_NA.db',
 'classapp_output\\data_RC.db',
 'classapp_output\\data_WZ.db',
 'classapp_output\\data_EM.db',
 'classapp_output\\data_FR.db',
 'classapp_output\\data_HC.db',
 'classapp_output\\data_JF.db',
 'classapp_output\\data_MG.db']

Creating a list of data frames from all users

In [5]:
data_frames = []

sql = '''
SELECT * FROM class_methods
'''
for db_path in db_paths:
    if os.path.exists(db_path):
        conn = sqlite3.connect(db_path)
        data_frames.append(pd.read_sql_query(sql, conn))
        conn.commit()
        conn.close()

data_frames

[    user  method_id                                        description  \
 0     GV          0  Prick the sweet potatoes two or three times wi...   
 1     GV          0  Prick the sweet potatoes two or three times wi...   
 2     GV          1  To make the smoked mackerel filling, mix all t...   
 3     GV          1  To make the smoked mackerel filling, mix all t...   
 4     GV          2  To make the pea and feta filling, microwave th...   
 ..   ...        ...                                                ...   
 125   GV         96  Pour in the beef stock and stir well until the...   
 126   GV         97  Transfer the vegetables and thickened stock to...   
 127   GV         97  Transfer the vegetables and thickened stock to...   
 128   GV         99  Arrange the potato slices on top of the hotpot...   
 129   GV        100  Bake the hotpot in the oven for 1-1� hours, or...   
 
      Measuring  Plating  Smoking  Toasting  Microwaving  Air Frying  \
 0            0        0  

Concatenating that list in one data frame

In [6]:
# concatenating
users_methods = pd.concat(data_frames)
users_methods.head(5)

,user,method_id,description,Measuring,Plating,Smoking,Toasting,Microwaving,Air Frying,Double Boiler,...,Seasoning,Salting,Slicing,Chopping Fruits,Chopping Mushroom,Chopping Herbs,Mincing,Batonnet,Dicing,Roughly Chopping
0,GV,0,Prick the sweet potatoes two or three times wi...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,GV,0,Prick the sweet potatoes two or three times wi...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GV,1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,GV,1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,GV,2,"To make the pea and feta filling, microwave th...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Saving description by id to later reinsertion 

In [7]:
description_by_method_id = users_methods[["method_id", "description"]].drop_duplicates().set_index('method_id')
description_by_method_id.head(10)

,description
method_id,
0,Prick the sweet potatoes two or three times wi...
1,"To make the smoked mackerel filling, mix all t..."
2,"To make the pea and feta filling, microwave th..."
3,"To make the veggie chilli filling, put the red..."
5,Whisk the egg whites in a clean large bowl usi...
6,Spread the meringue mixture into the prepared ...
7,"To make the lemon curd, put the butter into a ..."
8,Reserve 100g/3�oz (about 6 tablespoons) of the...
9,"To fill the roulade, mix the reserved lemon cu..."


Summing classifications for each value (to get multi-label classification)

In [8]:
vectorized_methods = users_methods.groupby('method_id').sum()
vectorized_methods.head(3)

,Measuring,Plating,Smoking,Toasting,Microwaving,Air Frying,Double Boiler,Bain Marie,Reducing,Water Bathing,...,Seasoning,Salting,Slicing,Chopping Fruits,Chopping Mushroom,Chopping Herbs,Mincing,Batonnet,Dicing,Roughly Chopping
method_id,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Joining the descriptions

In [9]:
classified_methods = pd.concat([description_by_method_id, vectorized_methods], axis='columns').reset_index(drop=True)
classified_methods.head(5)

,description,Measuring,Plating,Smoking,Toasting,Microwaving,Air Frying,Double Boiler,Bain Marie,Reducing,...,Seasoning,Salting,Slicing,Chopping Fruits,Chopping Mushroom,Chopping Herbs,Mincing,Batonnet,Dicing,Roughly Chopping
0,Prick the sweet potatoes two or three times wi...,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"To make the pea and feta filling, microwave th...",0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"To make the veggie chilli filling, put the red...",0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Whisk the egg whites in a clean large bowl usi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Listing the methods

In [10]:
techniques = list(classified_methods.columns[1:])
techniques

['Measuring',
 'Plating',
 'Smoking',
 'Toasting',
 'Microwaving',
 'Air Frying',
 'Double Boiler',
 'Bain Marie',
 'Reducing',
 'Water Bathing',
 'Deglazing',
 'Caramelising',
 'Poaching',
 'Simmering',
 'Boiling',
 'Steaming',
 'Stir Frying',
 'Deep Frying',
 'Flambing',
 'Braising',
 'Searing',
 'Sauteeing',
 'Grilling',
 'Pan Frying',
 'Grating',
 'Baking',
 'Roasting',
 'Squeezing',
 'Mashing',
 'Rehidrating',
 'Drying',
 'Kneading',
 'Tenderizing',
 'Whisking',
 'Mixing',
 'Blending',
 'Refrigerating',
 'Pickling',
 'Curing',
 'Freezing',
 'Infusing',
 'Marinating',
 'Seasoning',
 'Salting',
 'Slicing',
 'Chopping Fruits',
 'Chopping Mushroom',
 'Chopping Herbs',
 'Mincing',
 'Batonnet',
 'Dicing',
 'Roughly Chopping']

joining certain columns

In [11]:
joined_columns = {
    'Seasoning+': ['Salting', 'Seasoning'],
    'Pan Frying+': ['Pan Frying', 'Stir Frying', 'Grilling', 'Sauteeing'],
    'Boiling+': ['Boiling', 'Simmering', 'Reducing'],
    'Chopping+': ['Roughly Chopping', 'Slicing', 'Chopping Fruits', 'Chopping Mushroom', 'Chopping Herbs', 'Mincing', 'Batonnet', 'Dicing'],
}


joined_columns[list(joined_columns.keys())[0]]

['Salting', 'Seasoning']

In [12]:
for key in list(joined_columns.keys()):
    col = joined_columns[key]
    classified_methods[key] = classified_methods.loc[:, col].sum(axis=1)
    classified_methods.drop(columns=col, inplace=True)
classified_methods.head()

,description,Measuring,Plating,Smoking,Toasting,Microwaving,Air Frying,Double Boiler,Bain Marie,Water Bathing,...,Refrigerating,Pickling,Curing,Freezing,Infusing,Marinating,Seasoning+,Pan Frying+,Boiling+,Chopping+
0,Prick the sweet potatoes two or three times wi...,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,"To make the pea and feta filling, microwave th...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,"To make the veggie chilli filling, put the red...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Whisk the egg whites in a clean large bowl usi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


deleting unused columns

In [13]:
for column in classified_methods.columns:
    if set(classified_methods[column]) == {0}:
        classified_methods.drop(columns=[column], inplace=True)
classified_methods.head()

,description,Plating,Microwaving,Bain Marie,Grating,Baking,Roasting,Squeezing,Mashing,Kneading,Whisking,Mixing,Blending,Refrigerating,Marinating,Seasoning+,Pan Frying+,Boiling+,Chopping+
0,Prick the sweet potatoes two or three times wi...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,"To make the pea and feta filling, microwave th...",0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
3,"To make the veggie chilli filling, put the red...",0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,Whisk the egg whites in a clean large bowl usi...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Fixing anomalies in the table

In [14]:
#checking for anomalies (class > 1 or <0)
for column in classified_methods.columns[1:]:
    classified_methods.loc[classified_methods[column] > 1, column] = 1
    classified_methods.loc[classified_methods[column] < 0, column] = 0


In [15]:
classified_methods.head()

,description,Plating,Microwaving,Bain Marie,Grating,Baking,Roasting,Squeezing,Mashing,Kneading,Whisking,Mixing,Blending,Refrigerating,Marinating,Seasoning+,Pan Frying+,Boiling+,Chopping+
0,Prick the sweet potatoes two or three times wi...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"To make the smoked mackerel filling, mix all t...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,"To make the pea and feta filling, microwave th...",0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
3,"To make the veggie chilli filling, put the red...",0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,Whisk the egg whites in a clean large bowl usi...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Separating data in train and testing

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
df_train, df_test = train_test_split(classified_methods, test_size=0.2, random_state=42)

In [18]:
df_test

,description,Plating,Microwaving,Bain Marie,Grating,Baking,Roasting,Squeezing,Mashing,Kneading,Whisking,Mixing,Blending,Refrigerating,Marinating,Seasoning+,Pan Frying+,Boiling+,Chopping+
30,Spread the tomato mixture over the pizza bases...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,Prick the sweet potatoes two or three times wi...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,"For the pastry, place the flour, sugar and lem...",0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
31,Scatter with rocket or basil and serve immedia...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
18,"For the base, crush the gingernuts until they ...",0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
28,Prick one pizza base a few times with a fork. ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,"In a large bowl, whisk the egg yolks, vanilla ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
70,"Meanwhile, bring a saucepan of lightly salted ...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
4,Whisk the egg whites in a clean large bowl usi...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
12,Fill the tins equally with the mixture and bak...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


Pickling and csving the data frame for training use later

In [19]:
classified_methods.to_pickle(os.path.join('hand_classified_methods', 'classified_methods'))
classified_methods.to_csv(os.path.join('hand_classified_methods', 'classified_methods.csv'), index=False)
df_train.to_csv(os.path.join('hand_classified_methods', 'methods_train.csv'), index=False)
df_test.to_csv(os.path.join('hand_classified_methods', 'methods_test.csv'), index=False)